In [1]:
import json
import numpy as np
import pickle
import os


In [2]:
def budget_duration(budget_size):
    return int(4.92 * budget_size)

In [3]:
from collections import Counter
from functools import reduce
import itertools


dirs = sorted([
    "assamese_female_english",
    "hindi_male_english",
    "manipuri_female_english",
    "rajasthani_male_english",
    "kannada_male_english",
    "tamil_male_english",
    "malayalam_male_english",
    "gujarati_female_english",
])

for accents in itertools.combinations(dirs, 2):
    budget = 150
    target = 20
    etaScale = 1.0
    fxn = "GCMI"
    accent_feature = "39"
    content_feature = "tf_idf_2gram"
    g = "accent"
    gq = "accent"
    qq = "accent"
    accent_sim = "cosine"
    content_sim = "cosine"
    GCMI_selected_lists = []

    for accent in accents:
        path = os.path.join(
            ".", f"{accent}", "all", f"budget_{budget}", 
            f"target_{target}", f"{fxn}_etaScale_{etaScale}",
            f"accent_{accent_feature}", f"content_{content_feature}",
            f"kernel_g={g}_gq={gq}_qq={qq}", f"accent_{accent_sim}",
            f"content_{content_sim}"
        )

        with open(os.path.join(path, "train.json")) as f:
            GCMI_selected_lists.append([json.loads(line.strip()) for line in f.readlines()])

    # print(GCMI_selected_lists[0][:10])
    # print(GCMI_selected_lists[1][:10])
    # print(len(GCMI_selected_lists))

    duration = budget_duration(budget)

    new_list = []

    for i, accent in enumerate(accents):
        selected_list = []
        alloc_dur = duration/len(accents)
        selected_dur = 0
        index = 0
        while selected_dur < alloc_dur:
            selected_list.append(GCMI_selected_lists[i][index])
            selected_dur += selected_list[-1]["duration"]
            # print(selected_list[-1])
            # print('\n\n')
            index+=1
        new_list.extend(selected_list)

    for accent in accents:
        new_path = os.path.join(
            ".", f"{accent}", "all", "seperate_GCMI", "-".join(sorted([_ for _ in accents if _ != accent])), f"budget_{budget}", 
            f"target_{target}", f"{fxn}_etaScale_{etaScale}",
            f"accent_{accent_feature}", f"content_{content_feature}",
            f"kernel_g={g}_gq={gq}_qq={qq}", f"accent_{accent_sim}",
            f"content_{content_sim}"
        )

        os.makedirs(new_path, exist_ok=True)

        with open(os.path.join(new_path, "train.json"), 'w') as file:
            for line in new_list:
                file.write(json.dumps(line))
                file.write("\n")

            total_count = len(new_list)
            total_duration = reduce(lambda x, y: x + y, [line["duration"] for line in new_list], 0)
            accent_sample_count, accent_sample_duration = 0, 0
            for item in new_list:
                if item["audio_filepath"].split("/")[-4] == accent:
                    accent_sample_count += 1
                    accent_sample_duration += item["duration"]
            total_selections = Counter(
                [item["audio_filepath"].split("/")[-4] for item in new_list]
            )
            
            print("\n subset computed .... \n")
            stats = "total selections: " + str(total_count)
            stats += "\ntotal duration: " + str(total_duration)
            stats += "\naccented selections: " + str(accent_sample_count)
            stats += "\naccented duration: " + str(accent_sample_duration)
            stats += "\nall selections: " + str(total_selections)
            print(stats)
            with open(os.path.join(new_path, "stats.txt"), "w") as f:
                f.write(stats)
        




 subset computed .... 

total selections: 102
total duration: 744.0217030000001
accented selections: 61
accented duration: 369.277685
all selections: Counter({'assamese_female_english': 61, 'gujarati_female_english': 41})

 subset computed .... 

total selections: 102
total duration: 744.0217030000001
accented selections: 41
accented duration: 374.7440179999999
all selections: Counter({'assamese_female_english': 61, 'gujarati_female_english': 41})

 subset computed .... 

total selections: 128
total duration: 742.523835
accented selections: 61
accented duration: 369.277685
all selections: Counter({'hindi_male_english': 65, 'assamese_female_english': 61, 'rajasthani_male_english': 2})

 subset computed .... 

total selections: 128
total duration: 742.523835
accented selections: 65
accented duration: 361.8963369999999
all selections: Counter({'hindi_male_english': 65, 'assamese_female_english': 61, 'rajasthani_male_english': 2})

 subset computed .... 

total selections: 123
total durat